In [4]:
from datasets import load_dataset

openbookqa = load_dataset("openbookqa", "additional")

Using the latest cached version of the module from /home/yoda/.cache/huggingface/modules/datasets_modules/datasets/openbookqa/f338ccacfbc86fb8c2de3aa1c06d2ce686933de3bca284dba97d32592c52b33f (last modified on Thu Aug  3 10:59:09 2023) since it couldn't be found locally at openbookqa., or remotely on the Hugging Face Hub.
Generating test split: 100%|████████████████████████████████████████████████| 500/500 [00:00<00:00, 10929.44 examples/s]


In [5]:
openbookqa_subset = openbookqa.filter(lambda row: (
    row["humanScore"] >= 1.0
    and row["clarity"] >= 1.5
    and row["question_stem"].strip().endswith('?')
))

Filter: 100%|███████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 34834.68 examples/s]


In [16]:
make_chatml = lambda name, role, content: dict(
    name=name, role=role, content=content,
)

system = lambda name, content: make_chatml(
    role="system",
    name=name,
    content=content,
)

situation = lambda content: system(name="situation", content=content)
thought = lambda content: system(name="thought", content=content)
information = lambda content: system(name="information", content=content)
me = lambda content, name=None: make_chatml(
    role="assistant",
    content=content,
    name=name,
)

person = lambda content, name=None: make_chatml(
    role="user",
    content=content,
    name=name,
)

def get_answer_text(choices, answer_key):
    for choice, label in zip(choices["text"], choices["label"]):
        if label == answer_key:
            return choice

def to_chatml(row):
    question = row["question_stem"]
    choices_data = row["choices"]
    fact = row["fact1"]
    answer_key = row["answerKey"]

    choices = "\n".join([
        f"{label}) {text}"
        for label, text
        in zip(choices_data["label"], choices_data["text"])
    ])

    siutation_text = (
        "A person is talking to an intelligent AI assistant."
        " The person is quizzing the AI assistant on miscellaneous questions"
        " in a multiple choice format."
        " The assistant first thinks carefully about the question and lays down its reasoning."
        " Then answers the question with the correct choice key and the right answer."
    )
    
    mcq = f"Question: {question}\nChoices:\n{choices}"
    thought_text = f"Reaon: {fact}"
    answer_text = get_answer_text(choices_data, answer_key)
    person_text = f"Question: {question}\nChoices:\n{choices}"
    me_text = f"The correct answer is: {answer_key}. {answer_text}"
    
    chatml = [
        situation(situation_text),
        person(person_text),
        thought(thought_text),
        me(me_text),
    ]
    
    return dict(chatml=chatml)

openbookqa_chatml = openbookqa_subset.map(
    to_chatml
).remove_columns(['id', 'question_stem', 'choices', 'answerKey', 'fact1', 'humanScore', 'clarity', 'turkIdAnonymized'])
print(openbookqa_chatml)

DatasetDict({
    train: Dataset({
        features: ['chatml'],
        num_rows: 1021
    })
    validation: Dataset({
        features: ['chatml'],
        num_rows: 82
    })
    test: Dataset({
        features: ['chatml'],
        num_rows: 106
    })
})


In [19]:
openbookqa_chatml.push_to_hub("diwank/openbookqa_chatml", private=True)

Creating parquet from Arrow format: 100%|████████████████████████████████████████████████| 2/2 [00:00<00:00, 473.61ba/s]

Creating parquet from Arrow format: 100%|███████████████████████████████████████████████| 1/1 [00:00<00:00, 1410.80ba/s]

Creating parquet from Arrow format: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 499.74ba/s]

Pushing dataset shards to the dataset hub: 100%|██████████████████████████████████████████| 1/1 [00:01<00:00,  1.43s/it]
